<a href="https://colab.research.google.com/github/takagiyuusuke/Q_TETRIS_lv2/blob/main/Q_miniTETRIS_I%E3%83%9F%E3%83%8E%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ★ 最初に実行すること

## 0. インポート・乱数シードの設定

In [ ]:
import random

import torch
import numpy as np


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

## 1. miniTETRISのロジック
横6マスのテトリスを定義する

In [77]:
import copy
import time

class Tetris:
  #各種ブロックの定義
  block1 = [[-1, 2, 1], [0, 2, 1], [-1, 3, 1], [0, 3, 1], [-0.5, 2.5]] #O
  block2 = [[0, 2, 2], [0, 3, 2], [0, 4, 2], [-1, 3, 2], [0, 3]] #T
  block3 = [[-1, 1, 3], [-1, 2, 3], [-1, 3, 3], [-1, 4, 3], [-0.5, 2.5]] #I
  block4 = [[0, 2, 4], [0, 3, 4], [0, 4, 4], [-1, 2, 4], [0, 3]] #J
  block5 = [[-1, 4, 5], [0, 2, 5], [0, 3, 5], [0, 4, 5], [0, 3]] #L
  block6 = [[-1, 2, 6], [-1, 3, 6], [0, 3, 6], [0, 4, 6], [0, 3]] #Z
  block7 = [[0, 2, 7], [0, 3, 7], [-1, 3, 7], [-1, 4, 7], [0, 3]] #S
  blocks =[block2,block2,block2,block2,block2,block2,block2]

  #ブロックの回転の定義
  roundblock=[[[[0,-1],[-1,0],[0,1]],[[-1,0],[0,1],[1,0]],[[0,1],[1,0],[0,-1]],[[1,0],[0,-1],[-1,0]]],
  [[[-0.5,-1.5],[-0.5,-0.5],[-0.5,0.5],[-0.5,1.5]],[[-1.5,0.5],[-0.5,0.5],[0.5,0.5],[1.5,0.5]],[[0.5,-1.5],[0.5,-0.5],[0.5,0.5],[0.5,1.5]],[[-1.5,-0.5],[-0.5,-0.5],[0.5,-0.5],[1.5,-0.5]]],
  [[[-1,-1],[0,-1],[0,1]],[[-1,1],[-1,0],[1,0]],[[0,-1],[0,1],[1,1]],[[-1,0],[1,0],[1,-1]]],
  [[[-1,1],[0,1],[0,-1]],[[1,0],[-1,0],[1,1]],[[0,-1],[0,1],[1,-1]],[[1,0],[-1,0],[-1,-1]]],
  [[[-1,-1],[-1,0],[0,1]],[[-1,1],[0,1],[1,0]],[[0,-1],[1,0],[1,1]],[[1,-1],[0,-1],[-1,0]]],
  [[[0,-1],[-1,0],[-1,1]],[[-1,0],[0,1],[1,1]],[[1,-1],[1,0],[0,1]],[[-1,-1],[0,-1],[1,0]]]]

  #ブロックを回転する際に移動する座標を定義(詳しくはSRS)
  SRS =[[[0,0],[0,1],[-1,1],[2,0],[2,1]],[[0,0],[0,-1],[-1,-1],[2,0],[2,-1]],[[0,0],[0,-1],[1,-1],[-2,0],[-2,-1]],[[0,0],[0,1],[1,1],[-2,0],[-2,1]]]
  SRSofI =[[[0,0],[0,-1],[0,2],[-2,-1],[1,2]],[[0,0],[0,-2],[0,1],[1,-2],[-2,1]],[[0,0],[0,1],[0,-2],[1,2],[-1,-2]],[[0,0],[0,2],[0,-1],[-1,2],[2,-1]]]

  '''
  盤面の情報などの情報の初期化を行う
  display:画面を表示するか否か
  '''
  def __init__(self, display: bool, interval = 0):
    self.display = display
    self.interval = interval
    self.reset_game()

  '''
  ゲームをリセットする
  '''
  def reset_game(self):
    self.location=[]  #現在の盤面
    self.moveblock=[] #現在落下中のブロック
    self.hold=[]      #ホールド中のブロック
    self.angle = 0    #落下中のブロックの角度(0,1,2,3)
    self.num=1        #落下中のブロックが何番目か
    self.lines = 0    #消したラインの数
    self.score = 0    #スコア
    self.level = 0    #レベル
    self.canhold = True  #ホールドできるか
    self.turn_right = [] #右回転した場合のブロック配置
    self.turn_left = []  #左回転した場合のブロック配置
    self.before_action = "" #前回の行動
    self.before_before_action = "" #前々回の行動
    self.downcount = 0 #強制的に落下させるときに使用
    self.nex =[random.choice(Tetris.blocks) for i in range(20)] #後におちるブロック


    self.setblock(copy.deepcopy(self.nex[0])) #最初のブロックをセットする

  '''
  ゲームの盤面を表示する
  '''
  def printboard(self, display_force = False):
      if display_force:
        pass
      elif not self.display:
        return
      board = "\n"
      board += ("⏹"*8+"     NEXT "+str(self.num+1) +"\n")
      for i in range(22):
          board += ("⏹")
          for j in range(6):
              ad = [l for l in self.location + self.moveblock if l[0] ==i and l[1]==j]
              if ad:
                  if ad[0][2] ==0:
                      board += ("💥")
                  else:
                      board += (chr(128996+ad[0][2]))
              else:
                  board += ("⬜")
          board += ("⏹  ")
          if i == 0 or i == 1:
              for j in range(1,5):
                  af = [l for l in self.nex[self.num%20] if l[0] ==i-1 and l[1]==j]
                  if af:
                      board += (chr(128996+af[0][2]))
                  else:
                      board += ("⬜")
          elif i == 3:
              board += (" 👆👆👆 "+str(self.num+2))
          elif i == 4 or i == 5:
              for j in range(1,5):
                  af = [l for l in self.nex[(self.num+1)%20] if l[0] ==i-5 and l[1]==j]
                  if af:
                      board += (chr(128996+af[0][2]))
                  else:
                      board += ("⬜")
          elif i == 7:
              board += (" 👆👆👆 "+str(self.num+3))
          elif i == 8 or i == 9:
              for j in range(1,5):
                  af = [l for l in self.nex[(self.num+2)%20] if l[0] ==i-9 and l[1]==j]
                  if af:
                      board += (chr(128996+af[0][2]))
                  else:
                      board += ("⬜")
          elif i == 12:
              board += (" LEVEL =>"+str(self.level))
          elif i ==14:
              board += (" LINES =>"+str(self.lines))
          elif i == 16:
              board += (" SCORE =>"+str(self.score))
          elif i == 18:
              board += (" -HOLD-")
          elif i == 19 or i == 20:
              for j in range(1,5):
                  af = [l for l in self.hold if l[0] ==i-20 and l[1]==j]
                  if af:
                      board += (chr(128996+af[0][2]))
                  else:
                      board += ("⬜")
          elif i == 21:
              board += (" ------")
          board += ("\n")
      board += ("⏹"*8)
      print(board)
      time.sleep(self.interval)

  '''
  引数に指定したブロックを盤面に追加する
  '''
  def setblock(self, block):
      self.moveblock.extend(block)
      self.angle = 0
      if self.num%20==13:
          self.nex =[(random.choice(Tetris.blocks) if i < 10 else self.nex[i]) for i in range(20)]
      elif self.num%20==3:
          self.nex =[(random.choice(Tetris.blocks) if i >= 10 else self.nex[i]) for i in range(20)]

  '''
  ブロックが落下しなくなったか判定する
  '''
  def stopfall(self):
      for i in self.moveblock:
          aa = [j for j in self.location if (j[0] ==i[0]+1 and j[1]==i[1])]
          if len(aa)>0 or i[0] >=21:
              return True
      return False

  '''
  一列そろっている箇所があれば消去する
  '''
  def removeline(self):
      lines_first = self.lines
      L = []
      max_colum = 0
      for i in range(22):
          a = [False for l in range(6)]
          for j in range(6):
              ab = [k for k in self.location if k[0]==i and k[1]==j]
              if len(ab) > 0:
                  a[j] = True
          if all(a):
              self.location = [l for l in self.location if l[0] != i]
              self.location.extend([[i,k,0] for k in range(6)])
              L.append(i)
              self.lines += 1
              if self.lines % 10 == 0:
                  self.level += 1

      if L:
              self.printboard()
              self.location = [l for l in self.location if l[0] not in L]
              for j in L:
                  for m in self.location:
                      if m[0] < j:
                          m[0] += 1
      lines_final = self.lines
      self.score += (lines_final-lines_first)**2*1000*(self.level+1)

  '''
  ブロックを右に移動させる
  '''
  def right(self):
      for j in self.moveblock:
          j[1] +=1
      self.printboard()

  '''
  ブロックを左に移動させる
  '''
  def left(self):
      for j in self.moveblock:
          j[1] -=1
      self.printboard()

  '''
  ブロックを下に動かせるか確かめたのちに下に移動させる
  '''
  def down(self):
      if self.stopfall():
            if self.gameover():
                return
            self.moveblock = [x for x in self.moveblock if len(x) != 2]
            self.location.extend(self.moveblock)
            self.location
            self.moveblock = []
            self.removeline()
            self.setblock(copy.deepcopy(self.nex[self.num%20]))
            self.canhold = True
            self.num += 1
      else:
          for j in self.moveblock:
              j[0] +=1
          self.printboard()

  '''
  ブロックを回転させられるか確認する。
  - direction:右か左か
  '''
  def turn(self, direction):
      col = self.moveblock[0][2]
      angle_before = self.angle
      if direction == "left":
          self.angle = self.angle-1 if self.angle >0 else 3
      elif direction == "right":
          self.angle = self.angle+1 if self.angle <3 else 0
      if col > 1:
          e = Tetris.roundblock[col-2][self.angle]
          if col != 3:
              SRSn = (3 if angle_before == 1 else 2)if angle_before%2==1 else(1 if self.angle == 1 else 0)
              n = 0
              while True:
                  m = [[self.moveblock[4][0]+e[0][0]+Tetris.SRS[SRSn][n][0],self.moveblock[4][1]+e[0][1]+Tetris.SRS[SRSn][n][1]],
                                  [self.moveblock[4][0]+e[1][0]+Tetris.SRS[SRSn][n][0],self.moveblock[4][1]+e[1][1]+Tetris.SRS[SRSn][n][1]],
                                  [self.moveblock[4][0]+e[2][0]+Tetris.SRS[SRSn][n][0],self.moveblock[4][1]+e[2][1]+Tetris.SRS[SRSn][n][1]],
                                  [self.moveblock[4][0]+Tetris.SRS[SRSn][n][0],self.moveblock[4][1]+Tetris.SRS[SRSn][n][1]]]
                  ac = [j for j in self.location if (j[0] ==m[0][0] and j[1] ==m[0][1])or(j[0] ==m[1][0] and j[1] ==m[1][1])or
                      (j[0] ==m[2][0] and j[1] ==m[2][1])or(j[0] ==m[3][0] and j[1] ==m[3][1])]

                  if len(ac) ==0 and max(m[0][1],m[1][1],m[2][1],m[3][1]) <= 5 and min(m[0][1],m[1][1],m[2][1],m[3][1])>=0 and max(m[0][0],m[1][0],m[2][0],m[3][0])<=21:
                      return [[m[0][0],m[0][1],col],[m[1][0],m[1][1],col],[m[2][0],m[2][1],col],[m[3][0],m[3][1],col],m[3]]
                      break
                  elif n==4:
                      break
                  else:
                      n += 1
          elif col ==3:
              if angle_before == 0:
                  if self.angle == 1:
                      SRSn = 1
                  else:
                      SRSn = 0
              elif angle_before == 1:
                  if self.angle ==0:
                      SRSn = 3
                  else:
                      SRSn = 0
              elif angle_before == 2:
                  if self.angle == 1:
                      SRSn = 2
                  else:
                      SRSn = 3
              else:
                  if self.angle ==0:
                      SRSn = 2
                  else:
                      SRSn = 1
              n = 0
              while True:
                  m = [[round(self.moveblock[4][0]+e[0][0]+Tetris.SRSofI[SRSn][n][0]),round(self.moveblock[4][1]+e[0][1]+Tetris.SRSofI[SRSn][n][1])],
                                  [round(self.moveblock[4][0]+e[1][0]+Tetris.SRSofI[SRSn][n][0]),round(self.moveblock[4][1]+e[1][1]+Tetris.SRSofI[SRSn][n][1])],
                                  [round(self.moveblock[4][0]+e[2][0]+Tetris.SRSofI[SRSn][n][0]),round(self.moveblock[4][1]+e[2][1]+Tetris.SRSofI[SRSn][n][1])],
                                  [round(self.moveblock[4][0]+e[3][0]+Tetris.SRSofI[SRSn][n][0]),round(self.moveblock[4][1]+e[3][1]+Tetris.SRSofI[SRSn][n][1])]]
                  ac = [j for j in self.location if (j[0] ==m[0][0] and j[1] ==m[0][1])or(j[0] ==m[1][0] and j[1] ==m[1][1])or
                      (j[0] ==m[2][0] and j[1] ==m[2][1])or(j[0] ==m[3][0] and j[1] ==m[3][1])]

                  if len(ac) ==0 and max(m[0][1],m[1][1],m[2][1],m[3][1])<=5 and min(m[0][1],m[1][1],m[2][1],m[3][1])>=0 and max(m[0][0],m[1][0],m[2][0],m[3][0])<=21:
                      return [[m[0][0],m[0][1],col],[m[1][0],m[1][1],col],[m[2][0],m[2][1],col],[m[3][0],m[3][1],col],[self.moveblock[4][0]+Tetris.SRSofI[SRSn][n][0],self.moveblock[4][1]+Tetris.SRSofI[SRSn][n][1]]]
                      break
                  elif n==4:
                      break
                  else:
                      n += 1
      return []

  def turnright(self):
      self.moveblock = self.turn_right
      self.printboard()

  def turnleft(self):
      self.moveblock = self.turn_left
      self.printboard()

  '''
  ブロックをホールドする
  '''
  def holding(self):
      if not self.hold:
          self.hold =Tetris.blocks[self.moveblock[0][2]-1][:]
          self.moveblock = []
          self.setblock(copy.deepcopy(self.nex[self.num%20]))
          self.canhold = True
          self.num += 1
      else:
          if self.canhold:
              self.hold , self.moveblock = copy.deepcopy(Tetris.blocks[self.moveblock[0][2]-1]) , copy.deepcopy(self.hold)
              self.canhold = False
          else:
              return
      self.printboard()

  '''
  ゲームオーバーかどうかの判定を行う
  '''
  def gameover(self):
    try:
      min_moveblock = min(self.moveblock[i][0] for i in range(len(self.moveblock)))
    except:
      min_moveblock = 22
    try:
      min_location = min(self.location[i][0] for i in range(len(self.location)))
    except:
      min_location = 22

    return min_moveblock < 2 and min_location < 2

  '''
  盤面と落下中のブロック、ホールドブロックの形状を返す
  '''
  def get_state(self):
    # 上部の形状のみを抽出して返す関数
    if self.location:
      mine0 = min(self.location[i][0] for i in range(len(self.location)))
    else:
      mine0 = 22
    def transform_matrix(matrix, mine0):
      result = np.zeros(6, dtype=int)
      for j in range(6):
        try:
          mine0_ = min(self.location[i][0] for i in range(len(self.location)) if self.location[i][1] == j)
        except:
          mine0_ = 22
        result[j] += ((mine0_ - mine0) if (mine0_ - mine0) <= 3 else 4)
      return result
    height = mine0 - int(self.moveblock[4][0])
    location = tuple(transform_matrix(self.location, mine0))
    moveblock = self.moveblock[0][2], self.angle, min(height, 4) , int(self.moveblock[4][1]) #, self.hold[0][2] if self.hold else self.nex[self.num%20][0][2]
    return location, moveblock

  '''
  すべての可能な行動を返す
  - 状態のループを防止するために以下の行動を追加で禁止
  ・右移動→左移動→右移動
  ・左移動→右移動→左移動
  ・右回転→左回転→右回転
  ・左回転→右回転→左回転
  '''
  def get_possible_actions(self):
    possible_actions = ["down"]
    right = True
    for i in self.moveblock:
        aa = [j for j in self.location if (j[1] ==i[1]+1 and j[0]==i[0])]
        if len(aa) > 0 or i[1] >= 5:
            right = False
            break
    if (right):
      if not (self.before_action == "left"  and self.before_before_action == "right"):
        possible_actions.append("right")
    left = True
    for i in self.moveblock:
        aa = [j for j in self.location if (j[1] ==i[1]-1 and j[0]==i[0])]
        if len(aa) > 0 or i[1] <=0:
            left = False
            break
    if (left):
      if not (self.before_action == "right" and self.before_before_action == "left"):
        possible_actions.append("left")
    self.turn_left = self.turn("left")
    self.turn_right = self.turn("right")
    if (self.turn_left):
      if not (self.before_action == "turn-right" and self.before_before_action == "turn-left"):
        possible_actions.append("turn-left")
    if (self.turn_right):
      if not (self.before_action == "turn-left" and self.before_before_action == "turn-right"):
        possible_actions.append("turn-right")
    if self.canhold:
      possible_actions.append("hold")
    random.shuffle(possible_actions)
    return possible_actions #シャッフルして渡すことで学習初期の学習効率を向上させる

  '''
  スコアを取得する
  '''
  def get_score(self):
    return self.score

  '''
  盤面の変更を行う
  '''
  def modify_board(self, action):
    # self.score -= 1
    self.before_before_action = self.before_action
    self.before_action = action

    if action == "right":
        self.right()
    elif action == "left":
        self.left()
    elif action == "down":
        self.down()
    elif action == "turn-left":
        self.turnleft()
        # self.score -= 10
    elif action == "turn-right":
        self.turnright()
        # self.score -= 10
    elif action == "hold":
        self.holding()
    # 20回の操作ごとに強制落下させる
    if action == "down":
      self.downcount = 0
    else:
      self.score -= 1
      self.downcount += 1
      if self.downcount >= 20:
        self.down()
        self.downcount = 0


## 2. Agentの定義

In [ ]:
class Agent:
    def __init__(self):
        self.frozen = False

    def train(self):
        self.frozen = False

    def eval(self):
        self.frozen = True

    def _observe(self, tetris: Tetris ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        return tetris.get_state()

    def action(self, tetris: Tetris):
        pass

    def update(self, tetris: Tetris, state, action, reward1, new_state):
        pass

### 2-1. Qエージェントの定義

In [ ]:
class QAgent(Agent):
    def __init__(self, lr: float, eps = 0.3):
        super().__init__()
        self.q_table = {}  # Qテーブルの初期化
        self.discount_factor = 0.85
        self.learning_rate = lr
        self.epsilon = eps

    def action(self, tetris: Tetris) :
        board, block = self._observe(tetris)
        possible_actions = tetris.get_possible_actions()
        if not self.frozen and random.random() < self.epsilon:
            return random.choice(possible_actions)
        else:
            best_action, _ = self._get_the_best(board, block, possible_actions)
            return best_action

    def _get_the_best(self, board, block, possible_moves):
        best_move = None
        best_q_value = -float('inf')
        for move in possible_moves:
            q_value = self.q_table.get((board,block,move), 0)
            if q_value > best_q_value:
                best_q_value = q_value
                best_move = move
        return best_move, best_q_value

    #stateとnext_stateはboardとblockを渡す
    def update(self, tetris, state, action, reward, next_state):
        if self.frozen:
            return None

        old_value = self.q_table.get((state[0], state[1], action), 0)
        possible_moves = tetris.get_possible_actions()

        assert next_state is not None
        next_max = max([self.q_table.get((next_state[0], next_state[1], next_action), 0) for next_action in possible_moves])

        new_value = old_value + self.learning_rate * (reward + self.discount_factor * next_max - old_value)
        self.q_table[(state[0], state[1], action)] = new_value

### 2-2. 人間エージェントの定義

In [ ]:
class HumanAgent(Agent):
  def action(self, tetris: Tetris):
        valid_moves = tetris.get_possible_actions()
        while True:
            user_input = input("Enter your move: ")
            return user_input

## 3. Env.

In [69]:
import tqdm
import plotly.graph_objects as go

class Env:
    def __init__(self, agent: Agent, tetris: Tetris) -> None:
        self.agent = agent
        self.tetris = tetris

    def _get_reward(self) -> int:
        score = self.tetris.get_score()
        return score

    def train(self, episodes: int, visualize=False) :  # Nエピソード実行
        record = []
        for i in tqdm.tqdm(range(episodes)):
            score = self.execute(train=True, visualize=visualize)
            record.append(self._get_reward())
        print("Report:")
        return record

    def execute(self, train=False, visualize=True):  # 1 episode
        self.tetris.reset_game()

        while not self.tetris.gameover():
            state = self.tetris.get_state()
            while True:
                action = self.agent.action(self.tetris)
                before_point = self.tetris.get_score()
                self.tetris.modify_board(action)
                after_point = self.tetris.get_score()
                reward = after_point - before_point
                if train:
                    next_state = self.tetris.get_state()

                    if action is not None:
                        self.agent.update(self.tetris, state, action, reward, next_state)
                break

        # game end
        score = self.tetris.get_score()
        if random.random() < 0.001:
          self.tetris.printboard(display_force = True)
        return score

# 4. Q学習
学習中、現状確認のために一定の確率でゲームオーバー時の盤面が出力される

In [ ]:
q_agent = QAgent(lr=0.1, eps=0.1)
q_agent.epsilon = 0.3 #εはこのくらい大きめの方が経験的に良い
tetris = Tetris(False)
env = Env(q_agent, tetris)
record = env.train(1000)
fig = go.Figure(data=go.Scatter(y=record))
fig.show()

# 5. 学習が完了したQエージェントにプレイさせてみる
モデルが実際にどのようなプレイをするのか確かめる。

In [ ]:
q_agent.eval()
env = Env(q_agent, Tetris(True, interval = 0.05))
env.execute()
q_agent.train()

# 6. 追加で学習させたい場合
モデルをリセットせずにさらに学習させたい場合はここを実行  
epsilon, lr を適宜変更するとよい

In [ ]:
# @title 追加学習の内容 { display-mode: "both" }
epsilon = 0.3 # @param {type:"number"}
lr = 0.1 # @param {type:"number"}
train_num = 1000 # @param {type:"number"}
q_agent.train()
tetris = Tetris(False)
q_agent.epsilon = epsilon
q_agent.learning_rate = lr
env = Env(q_agent, tetris)
record = env.train(train_num)
fig = go.Figure(data=go.Scatter(y=record))
fig.show()

# 付録

## 手動で遊んでみる
※ 非常にユーザビリティが悪いため、やらない方がよい(笑)


In [ ]:
env = Env(HumanAgent(), Tetris(True))
env.execute()

## Qテーブルを見てみる
モデルが学習したQ値を見ることができる。  
ただし、モデルが重い場合は非常に時間がかかったり、そもそも出力してくれなかったりする可能性がある


In [ ]:
print((q_agent.q_table))

## うまく学習できた場合のデータを用いてプレイ
学習にはランダム性があるので失敗することがある  
うまくいった際のQテーブルを用いたプレイはこちらから  
※約10000エピソード程度の学習結果

In [ ]:
# @title うまくいったQテーブルをgithubから読み込む { display-mode: "form" }

!git clone https://github.com/takagiyuusuke/Q_TETRIS_lv2.git

import os
import sys

os.chdir('Q_TETRIS_lv2')
sys.path.append(os.getcwd())

import GoodQTable

good_q_table = GoodQTable.good_q_table

In [ ]:
q_agent = QAgent(lr = 0.1)
q_agent.q_table = good_q_table
q_agent.eval()
interval = 0.05 #intervalの値は再生の速さ(小さい値ほど早い)
assert interval >= 0
env = Env(q_agent, Tetris(True,interval = interval))
env.execute()